In [2]:
from dataretrieval import nwis
from dataretrieval import wqp
import pandas as pd

# NWM dataset 
# https://github.com/NOAA-OWP/hydrotools?tab=readme-ov-file


### Data source, availability, and information
From upstream to downsteam

#### 3. Baker River at Henry Thompson BR at Concrete, WA - 12193400 
Note: inflow discharge B\
source: https://waterdata.usgs.gov/monitoring-location/12193400/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, discharge\
Latitude 48°32'26",   Longitude 121°44'32"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110005\
Drainage area: 297 square miles\
Datum of gage: 174.08 feet above   NAVD88.

#### 4. Skagit River Near Concrete, WA - 12194000
Note: inflow discharge C\
source: https://waterdata.usgs.gov/monitoring-location/12194000/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, discharge\
Latitude 48°31'28",   Longitude 121°46'11"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110007\
Drainage area: 2,737 square miles\
Datum of gage: 133.96 feet above   NAVD88.



In [3]:
def get_nWIS_data( siteNumbers, parameterCds, startDate, endDate):
    # This function download NWIS data
    # and save it to a CSV file

    for site in siteNumbers:
    
        # Get site information
        chop_tank_info, md = nwis.get_info(sites=site)

        # print site information
        print(f"Site information for {site}:")
        csv_filename = f"info_site_{site}.csv"
        chop_tank_info.to_csv(csv_filename)

        #rawData, md = nwis.get_dv(sites=siteNumbers, parameterCd=parameterCds,
        #                                     start=startDate, end=endDate)
        rawData, md = nwis.get_iv(sites=site, parameterCd=parameterCds,
                                start=startDate, end=endDate)
        
        # Check if columns exist before renaming
        rename_mapping = {
            "00060": "Discharge (cfs)",
            "00065": "Gage Height (ft)",
            "00010": "Temperature (C)",
            "00062": "WSE above datum (ft)",
        }

        # Check if the columns to rename exist in the DataFrame
        # and rename them accordingly 
        existing_columns = rawData.columns.intersection(rename_mapping.keys())
        if not existing_columns.empty:
            rawData = rawData.rename(columns={col: rename_mapping[col] for col in existing_columns})
            # Drop columns ending with '_cd'
            columns_to_drop = [col + '_cd' for col in existing_columns]
            rawData = rawData.drop(columns=columns_to_drop)
        
        # Remove timezone from datetime and convert to local time
        rawData=rawData.tz_localize(None)
        rawData.index=rawData.index-pd.Timedelta(hours=8)
        
        # Convert index to a column
        rawData = rawData.reset_index().rename(columns={'datetime': 'date'})
        
        # # Save rawData to a CSV file
        # csv_filename = f"rawData_site_{site}.csv"
        # rawData.to_csv(csv_filename, index=False)
        # print(f"Data for site {site} saved to {csv_filename}")
        # print(rawData.head(10))
    return rawData
   

In [4]:
# site numbers for the USGS gage
siteNumbers1 = ["12193400"]
siteNumbers2 = ["12194000"]
#siteNumbers = ["12193400"]

parameterCds = ["00065","00060","00010","00062"]  # wse, discharge, and temperature, wse above datum
#statCd = ["00001","00003"]  # Mean and maximum
startDate = "2021-01-01"
endDate = "2023-01-02"

rawData1=get_nWIS_data( siteNumbers1, parameterCds, startDate, endDate)
rawData1=rawData1.drop(columns=['site_no'])
rawData2=get_nWIS_data( siteNumbers2, parameterCds, startDate, endDate)
rawData2=rawData2.drop(columns=['site_no'])

Site information for 12193400:
Site information for 12194000:


In [5]:
# Merge rawData1 with rawData2 on the 'date' column using an outer join
merged_rawData1 = pd.merge(rawData2[['date']], rawData1, on='date', how='left')

# Set the 'date' column as the index for interpolation
merged_rawData1.set_index('date', inplace=True)

# Interpolate missing values in rawData1 columns to match rawData2's date column
interp_rawData1 = merged_rawData1.interpolate(method='time')

# Reset the index after interpolation
interp_rawData1.reset_index(inplace=True)

# Print or save the interpolated data
print(interp_rawData1.head())


                 date  Discharge (cfs)  Gage Height (ft)
0 2021-01-01 00:00:00      1680.000000          3.190000
1 2021-01-01 00:05:00      1640.000000          3.160000
2 2021-01-01 00:10:00      1600.000000          3.130000
3 2021-01-01 00:15:00      1560.000000          3.100000
4 2021-01-01 00:20:00      1516.666667          3.063333


In [6]:
# Ensure the 'date' column is excluded from the addition operation
numeric_columns = interp_rawData1.columns.difference(['date'])

# Add only numeric columns
rawData3 = interp_rawData1.copy()
rawData3[numeric_columns] = interp_rawData1[numeric_columns] + rawData2[numeric_columns]

# Add calculated columns for discharge in cms and gage height in meters for rawData3
rawData3['Discharge (cms)'] = rawData3['Discharge (cfs)'] * 0.0283168  # Convert cfs to cms
rawData3['Gage Height (m)'] = rawData3['Gage Height (ft)'] * 0.3048  

# Print the first 10 rows of the resulting DataFrame
print(rawData3.head(10))

# Save the merged data to a CSV file
csv_filename = "rawDataUpstream.csv"
rawData3.to_csv(csv_filename, index=False)
print(f"Merged data saved to {csv_filename}")

                 date  Discharge (cfs)  Gage Height (ft)  Discharge (cms)  \
0 2021-01-01 00:00:00     18280.000000         21.730000       517.631104   
1 2021-01-01 00:05:00     17940.000000         21.640000       508.003392   
2 2021-01-01 00:10:00     17900.000000         21.600000       506.870720   
3 2021-01-01 00:15:00     17760.000000         21.550000       502.906368   
4 2021-01-01 00:20:00     17816.666667         21.533333       504.510987   
5 2021-01-01 00:25:00     17673.333333         21.456667       500.452245   
6 2021-01-01 00:30:00     17530.000000         21.390000       496.393504   
7 2021-01-01 00:35:00     17623.333333         21.416667       499.036405   
8 2021-01-01 00:40:00     17416.666667         21.373333       493.184267   
9 2021-01-01 00:45:00     17410.000000         21.350000       492.995488   

   Gage Height (m)  
0         6.623304  
1         6.595872  
2         6.583680  
3         6.568440  
4         6.563360  
5         6.539992  
6    